# ResNet 구현
- 핵심: input 과 output을 더해준다
- input을 기억했다가 output과 더해야 하니 Sequential 방식으로는 모델 구현이 불가능하다.

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, Add

In [5]:
np.random.seed(7777)
tf.random.set_seed(7777)

## Functional API
- `tf.keras.Sequential` 보다 더 유연하게 모델을 정의 가능

In [18]:
## Functional API 를 이용해 ResNet 구현
def build_resnet(input_shape):
    inputs = Input(input_shape)

    net = Conv2D(32, kernel_size=3, strides=2,
                 padding='same', activation='relu')(inputs)
    net = MaxPool2D()(net) # input
    
    net1 = Conv2D(64, kernel_size=1, padding='same', activation='relu')(net)
    net2 = Conv2D(64, kernel_size=3, padding='same', activation='relu')(net1)
    net3 = Conv2D(64, kernel_size=1, padding='same', activation='relu')(net2)
    
    net1_1 = Conv2D(64, kernel_size=1, padding='same')(net) # input을 output과 더하기 위해 사이즈를 64로 맞춰줌
    net = Add()([net1_1, net3])  # input + output // 다시 input
    
    net1 = Conv2D(64, kernel_size=1, padding='same', activation='relu')(net)
    net2 = Conv2D(64, kernel_size=3, padding='same', activation='relu')(net1)
    net3 = Conv2D(64, kernel_size=1, padding='same', activation='relu')(net2)
    
    net = Add()([net, net3])
    
    net = MaxPool2D()(net)
    
    net = Flatten()(net)
    net = Dense(10, activation="softmax")(net)

    model = tf.keras.Model(inputs=inputs, outputs=net, name='resnet')
    
    return model

In [19]:
model = build_resnet((32, 32, 3))
model.summary()

Model: "resnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_37 (Conv2D)             (None, 16, 16, 32)   896         ['input_7[0][0]']                
                                                                                                  
 max_pooling2d_8 (MaxPooling2D)  (None, 8, 8, 32)    0           ['conv2d_37[0][0]']              
                                                                                                  
 conv2d_38 (Conv2D)             (None, 8, 8, 64)     2112        ['max_pooling2d_8[0][0]']        
                                                                                             

## Cifar10 dataset을 이용해 학습

In [20]:
class Cifar10DataLoader():
    def __init__(self):
        # data load
        (self.train_x, self.train_y), \
            (self.test_x, self.test_y) = tf.keras.datasets.cifar10.load_data()
        self.input_shape = self.train_x.shape[1:]

    def scale(self, x):

        return (x / 255.0).astype(np.float32)

    def preprocess_dataset(self, dataset):

        (feature, target) = dataset

        # scaling #
        scaled_x = np.array([self.scale(x) for x in feature])

        # label encoding #
        ohe_y = np.array([tf.keras.utils.to_categorical(
            y, num_classes=10) for y in target])
        
        return scaled_x, ohe_y.squeeze(1)

    def get_train_dataset(self):
        return self.preprocess_dataset((self.train_x, self.train_y))

    def get_test_dataset(self):
        return self.preprocess_dataset((self.test_x, self.test_y))

In [21]:
cifar10_loader = Cifar10DataLoader()

train_x, train_y = cifar10_loader.get_train_dataset()

print(train_x.shape, train_x.dtype)
print(train_y.shape, train_y.dtype)

test_x, test_y = cifar10_loader.get_test_dataset()

print(test_x.shape, test_x.dtype)
print(test_y.shape, test_y.dtype)

170498071/170498071 [==============================] - 12s 0us/step
(50000, 32, 32, 3) float32
(50000, 10) float32
(10000, 32, 32, 3) float32
(10000, 10) float32


In [22]:
lr = 0.03
opt = tf.keras.optimizers.Adam(learning_rate=lr)
loss = tf.keras.losses.categorical_crossentropy

model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [23]:
hist = model.fit(train_x, train_y, epochs=10, batch_size = 128, validation_data=(test_x, test_y))

Epoch 1/10
391/391 [==============================] - 60s 146ms/step - loss: 2.4970 - accuracy: 0.0979 - val_loss: 2.4042 - val_accuracy: 0.1000
Epoch 2/10
391/391 [==============================] - 44s 111ms/step - loss: 2.3254 - accuracy: 0.1021 - val_loss: 2.3439 - val_accuracy: 0.1000
Epoch 3/10
391/391 [==============================] - 48s 123ms/step - loss: 2.3214 - accuracy: 0.0994 - val_loss: 2.3917 - val_accuracy: 0.1000
Epoch 4/10
391/391 [==============================] - 48s 123ms/step - loss: 2.3143 - accuracy: 0.0997 - val_loss: 2.3184 - val_accuracy: 0.1000
Epoch 5/10
391/391 [==============================] - 44s 113ms/step - loss: 2.3159 - accuracy: 0.1008 - val_loss: 2.3172 - val_accuracy: 0.1000
Epoch 6/10
391/391 [==============================] - 43s 110ms/step - loss: 2.3139 - accuracy: 0.1007 - val_loss: 2.3459 - val_accuracy: 0.1000
Epoch 7/10
391/391 [==============================] - 43s 111ms/step - loss: 2.3185 - accuracy: 0.1003 - val_loss: 2.3301 - val_ac